# Managing custom Jupyter environments with Singularity
---
In this tutorial we will cover the basic workflow for managing custom software environments for Jupyter Notebooks using Singularity.

## Pull the base image from SingularityHub
We maintain a base image on SingularityHub for running Jupyter* (https://singularity-hub.org/collections/440/). This image contains the minimum dependencies and configuration needed to run containerized Notebooks (standalone or JupyterHub-spawned), and is intended to serve as a base for user-built software environments.

### A basic pull
Start by pulling the Jupyter base image from SingularityHub:

In [1]:
singularity pull --name "jupyter.img" shub://ResearchComputing/singularity-jupyter:master

Progress |===================================| 100.0% 


Done. Container is at: ./jupyter.img


There it is! Your container is good to go.

In [2]:
singularity exec jupyter.img jupyter -h

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

optional arguments:
  -h, --help     show this help message and exit
  --version      show the jupyter command's version and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json

Available subcommands: bundlerextension kernelspec migrate nbconvert
nbextension notebook run serverextension troubleshoot trust


## Customizing the base image
The base image we provide is meant to capture the _minimum_ config and dependencies to run Jupyter Notebooks. Here we detail how to customize the base image to better suit your needs.

### Resize the image
This image uses the default size set by Singularity _(768M)_ which is great for quick builds and pulls, but it is likely you'll need more space to accommodate your custom software stack.

In [3]:
ls -lsah | grep jupyter.img

769M -rwxrwxr-x.  1 vagrant vagrant 769M Sep 26 23:05 jupyter.img


To increase the size of your image you must create a new image, and then import the Jupyter base image:

In [4]:
singularity create --size 1500 jupyter-ext.img
singularity export jupyter.img > singularity import jupyter-ext.img

Initializing Singularity image subsystem
Opening image file: jupyter-ext.img
Creating 1500MiB image
Binding image to loop
Creating file system within image
Image is done: jupyter-ext.img


### Installing software _(the quick way)_
By default Singularity containers mounted as read-only volumes, which means you won't be able to add content or install software _(even as a privileged user)_, save for default or system-mounted paths. In order to add content you must run your Singularity command with the `--writable` flag.

### Installing Software _(the reproducible way)_
Shelling into your container and making ad-hoc changes is excellent for debugging and initial development, but it is considered bad practice as the steps needed to construct your software environment are not captured and cannot be reproduced.

To make durable, reproducible changes you need to build a spec file from which you can bootstrap your container. Bootstrapping must be done by a privleged user

In [17]:
ls

configure-singularity-env.sh
jupyter.img
managing-custom-jupyter-envs-with-singularity.ipynb


# Experiment Space _!!!_

In [ ]:
singularity pull --name "jupyter-r.img" docker://jupyter/r-notebook

Initializing Singularity image subsystem
Opening image file: jupyter-spark.img
Creating 5709MiB image
Binding image to loop
Creating file system within image
Image is done: jupyter-spark.img
Docker image path: index.docker.io/jupyter/r-notebook:latest
Cache folder set to /home/sampedro/.singularity/docker
[18/22] |============================|------|  81.8% ERROR Error writing to /home/sampedro/.singularity/docker/sha256:ff2b77b87197934ca8d13f66e3a1036bbce12378416646fb6dc79de254199d66.tar.gz.EkyuJc.18htbW: [Errno 28] No space left on device exiting
ERROR Error downloading https://index.docker.io/v2/jupyter/r-notebook/blobs/sha256:ff2b77b87197934ca8d13f66e3a1036bbce12378416646fb6dc79de254199d66. Do you have permission to write to /home/sampedro/.singularity/docker?


In [2]:
rm -rf jupyter-spark.img
ls -lsa

total 1256116
     34 drwxr-sr-x.  3 sampedro sampedrogrp        213 Sep 26 15:39 .
     34 drwxrws---. 12 sampedro sampedrogrp        407 Sep 25 14:39 ..
     26 -rwxr-xr-x.  1 sampedro sampedrogrp        194 Sep 26 10:51 configure-singularity-env.sh
      2 drwxr-sr-x.  2 sampedro sampedrogrp         80 Sep 25 15:55 .ipynb_checkpoints
  73010 -rwxr-xr-x.  1 sampedro sampedrogrp 1572864032 Sep 26 10:53 jupyter-ext.img
1182938 -rwxr-xr-x.  1 sampedro sampedrogrp  805306400 Sep 26 15:00 jupyter.img
     74 -rw-r--r--.  1 sampedro sampedrogrp      20009 Sep 26 15:34 managing-custom-jupyter-envs-with-singularity.ipynb


In [11]:
singularity exec --writable jupyter.img echo hello

hello


In [6]:
df -h /home/vagrant

Filesystem                       Size  Used Avail Use% Mounted on
/dev/mapper/VolGroup00-LogVol00   38G  3.0G   35G   8% /


In [5]:
man df

DF(1)                            User Commands                           DF(1)



NAME
       df - report file system disk space usage

SYNOPSIS
       df [OPTION]... [FILE]...

DESCRIPTION
       This  manual  page  documents  the  GNU version of df.  df displays the
       amount of disk space available on the file system containing each  file
       name  argument.   If  no file name is given, the space available on all
       currently mounted file systems is shown.  Disk space  is  shown  in  1K
       blocks  by  default, unless the environment variable POSIXLY_CORRECT is
       set, in which case 512-byte blocks are used.

       If an argument is the absolute file name of a disk device node contain‐
       ing  a  mounted  file system, df shows the space available on that file
       system rather than on the file system containing the device node.  This
       version  of  df  cannot show the space available on unmounted file sys‐
       tems, because on most kinds of systems 